## Adding Libraries, Importing Dataset.

In [11]:
options(dplyr.summarise.inform = FALSE)
options(warn = -1)
library(tidyverse)
library(baseballr)
library(ggthemes) # Themes for ggplot2.
library(ggimage) # Supports images in ggplot2.
library(gt) # Create tables.
library(readr) # Read in datasets.
library(scatterplot3d)
library(reshape)
library(ExcelFunctionsR)

In [3]:
# Custom plot settings.
theme_reach <- function() {
  theme_fivethirtyeight() +
    theme(
      legend.position = "none",
      plot.title = element_text(size = 22, hjust = 0.5, face = "bold"),
      plot.subtitle = element_text(size = 18, hjust = 0.5),
      plot.caption = element_text(size = 16),
      axis.title.x = element_text(size=18, margin = margin(t = 20)),
      axis.title.y = element_text(size=18, margin = margin(r = 20)),
      axis.text = element_text(size = 14),
      strip.text = element_text(size = 16, face = "bold"),
      legend.text = element_text(size = 14)
    )
}

In [4]:
all_data_2021 <- read.csv(file = "C:/Users/chris/Documents/Datasets/BaseballR/2021/2021merged.csv")

## Filter Dataset and Divide into Pre/Post-8-21-2021 Subsets.

In [25]:
logan_all <- all_data_2021 %>%
  filter(pitcher == 669302)

logan_pre <- logan_all %>%
  filter(game_date.y <= "2021-08-21")
logan_post <- logan_all %>%
  filter(game_date.y > "2021-08-21")

## Compute ERA for Subsets.

In [26]:
logan_season_stats_pre <- logan_all %>%
  filter(game_date.y <= "2021-08-21") %>%
  group_by(game_date.y, at_bat_number) %>%
  summarize(RBI = mean(result.rbi),
            Outs = mean(count.outs.end - count.outs.start))
  
logan_season_stats_post <- logan_all %>%
  filter(game_date.y > "2021-08-21") %>%
  group_by(game_date.y, at_bat_number) %>%
  summarize(RBI = mean(result.rbi),
            Outs = mean(count.outs.end - count.outs.start))

inn_pre <- sum(logan_season_stats_pre$Outs / 3)
inn_post <- sum(logan_season_stats_post$Outs / 3)
era_pre <- round((47/inn_pre) * 9, 2)
era_post <- round((15/inn_post) * 9, 2)

## Compute and Arrange our Columns.

In [ ]:
logan_pre <- logan_pre %>%
  group_by(pitch_name) %>%
  summarize(pitch_count = n(),
            avg_x = round(mean(release_pos_x), 2),
            avg_delta_run_100 = round(mean(delta_run_exp), 4) * 100) %>%
  mutate(date_range = c("Through 8-21", "", "", "")) %>%
  mutate(era = c(era_pre, "", "", ""))

logan_post <- logan_post %>%
  group_by(pitch_name) %>%
  summarize(pitch_count = n(),
            avg_x = round(mean(release_pos_x), 2),
            avg_delta_run_100 = round(mean(delta_run_exp), 4) * 100) %>%
  mutate(date_range = c("After 8-21", "", "", "")) %>%
  mutate(era = c(era_post, "", "", ""))

logan_pre$delta_run_change <- c("", "", "", "")
logan_post$delta_run_change <- logan_post$avg_delta_run_100 - logan_pre$avg_delta_run_100

## Bind our Tables Vertically.

In [29]:
logan_table <- rbind(logan_pre, logan_post)
logan_table <- logan_table[, c(5, 6, 1, 2, 3, 4, 7)]
logan_table$delta_run_change <- round(as.numeric(logan_table$delta_run_change), 2)
logan_table

date_range,era,pitch_name,pitch_count,avg_x,avg_delta_run_100,delta_run_change
<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>
Through 8-21,5.16,4-Seam Fastball,896,-2.38,-0.86,NA
,,Changeup,114,-2.85,4.33,NA
,,Knuckle Curve,100,-1.91,0.48,NA
,,Slider,346,-2.67,1.11,NA
After 8-21,3.62,4-Seam Fastball,411,-1.29,-2.68,-1.82
,,Changeup,51,-1.76,1.44,-2.89
,,Knuckle Curve,45,-1.07,-1.14,-1.62
,,Slider,162,-1.06,3.52,2.41


## Create and Save Late-Season Adjustments Table.

In [31]:
logan_table_gt <- logan_table %>%
  gt() %>%
  cols_label(date_range = "Date Range",
             era = "ERA",
             pitch_name = "Pitch Name",
             avg_x = "Release Point X",
             avg_delta_run_100 = "\U0394 Runs Expected/100",
             delta_run_change = "Improvement",
             pitch_count = "Pitch Count") %>%
  cols_align(align = "center",
             columns = c(era, avg_x, avg_delta_run_100, delta_run_change, pitch_count)) %>%
  cols_width(c(era) ~ px(100)) %>%
  gtExtras::gt_theme_espn() %>%
  tab_header(title = "Logan Gilbert Late-Season Adjustments, 2021") %>%
  tab_style(style = cell_borders(sides = "right",
                                 color = "#D3D3D3",
                                 weight = px(2)),
            locations = cells_body(columns = c(era, pitch_count, avg_x),
                                   rows = everything())) %>%
  tab_style(style = cell_borders(sides = "top",
                                 color = "#808080",
                                 weight = px(4)),
            locations = cells_body(columns = everything(),
                                   rows = 1)) %>%
  tab_style(style = cell_borders(sides = "bottom",
                                 color = "#808080",
                                 weight = px(4)),
            locations = cells_body(columns = everything(),
                                   rows = 4)) %>%
  tab_style(style = cell_fill(color = "#ff726f"),
            locations = cells_body(columns = delta_run_change,
                                   rows = (delta_run_change > 0))) %>%
  tab_style(style = cell_fill(color = "#90EE90"),
            locations = cells_body(columns = delta_run_change,
                                   rows = (delta_run_change < 0)))

In [32]:
gtsave(logan_table_gt, "logan-overview-table.png")